# Random Ensemble Model

In [1]:
import sys
from os.path import abspath

sys.path.append(abspath('../src/'))

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from utils import *
from baseline_convnet import BaselineConvnet
import matplotlib.pyplot as plt
from sklearn import preprocessing
import random
from random_ensemble import RandomEnsemble

Using TensorFlow backend.


In [2]:
x_train, y_train, x_test, y_test, input_shape, num_classes = preprocess_mnist()
model = RandomEnsemble(input_shape=input_shape, num_classes=num_classes, n_proj=10, size_proj=8)


Loading mnist.
x_train shape: (60000, 28, 28, 1) 
x_test shape: (10000, 28, 28, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
classifier, projector = model.load_classifier(
    relative_path="../trained_models/baseline_convnet/baseline_proj10_size8/")

In [7]:
x_test_pred = model.evaluate_test(classifier, x_test, y_test)


Testing infos:
x_test.shape =  (10000, 28, 28, 1) 
y_test.shape =  (10000, 10) 


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Original test data.
Correctly classified: 9416
Incorrectly classified: 584
Test accuracy: 94.16%


In [8]:
x_test_fgsm, x_test_fgsm_pred = model.evaluate_adversaries(classifier, x_test, y_test)


Adversarial evaluation using FGSM method.

Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 4213
Incorrectly classified: 5787
Adversarial accuracy: 42.13%


In [9]:
x_test_deepfool, x_test_deepfool_pred = model.evaluate_adversaries(classifier, x_test, y_test, method='deepfool',
                                        adversaries_path='../data/mnist_x_test_deepfool.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 9303
Incorrectly classified: 697
Adversarial accuracy: 93.03%


In [10]:
x_test_projected, x_test_projected_pred = model.evaluate_adversaries(classifier, x_test, y_test, method='projected_gradient',
                                        adversaries_path='../data/mnist_x_test_projected_gradient.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 6314
Incorrectly classified: 3686
Adversarial accuracy: 63.14%


## Adversarial training

In [12]:
classifier, projector = model.load_classifier(
    relative_path="../trained_models/random_ensemble/robust_fgsm_proj10_size8/",
    model_name="random_ensemble_robust_fgsm")

In [13]:
x_test_fgsm, x_test_fgsm_pred = model.evaluate_adversaries(classifier, x_test, y_test, method='fgsm')


Adversarial evaluation using FGSM method.

Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 7123
Incorrectly classified: 2877
Adversarial accuracy: 71.23%


In [14]:
x_test_deepfool, x_test_deepfool_pred = model.evaluate_adversaries(classifier, x_test, y_test, 
                                                                     method='deepfool',
                                                                     adversaries_path='../data/mnist_x_test_deepfool.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 9130
Incorrectly classified: 870
Adversarial accuracy: 91.30%


In [15]:
x_test_projected, x_test_projected_pred = model.evaluate_adversaries(classifier, x_test, y_test, method='projected_gradient',
                                        adversaries_path='../data/mnist_x_test_projected_gradient.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 7657
Incorrectly classified: 2343
Adversarial accuracy: 76.57%
